In [ ]:
from extract.common import credentials
from extract.common import config

In [ ]:
import pymongo
from bson.json_util import dumps

host = config()['mongodb']['host']
user = credentials()['mongodb']['user']
password = credentials()['mongodb']['password']

class SaveProjects(object):
    def __init__(self):
        uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
        client = pymongo.MongoClient(uri)
        self._db = client[config()['mongodb']['db']['name']]
    
    def save(self, projects):
        self._db.project.insert_many(projects)

In [ ]:
save_project = SaveProjects()

In [ ]:
save_project._db.project.delete_many({})

In [ ]:
save_project._db.project.estimated_document_count()
[i for i in save_project._db.project.find()]

In [ ]:
save_project.save([{"a":1},{"b":2}])

In [ ]:
10000 + 548.99

In [ ]:
time = 119
capital = 12000
yield_ = 0.18
YEAR = 360
MINIMUN = 1000
def investment_return(capital, time, yield_):
    if capital % MINIMUN != 0:
        raise ValueError("capital must be multiple of 1000")
    revenue = (capital * yield_ * time)/YEAR
    commission_percentage = 0.077328
    commission = revenue * commission_percentage
    return_ = revenue - commission
    return {
        "revenue": revenue,
        "commission_percentage": commission_percentage,
        "commission": commission,
        "return": return_
    }
def fix_capital(capital):
    if (capital % MINIMUN) == 0:
        return 0
    capital = MINIMUN - (capital % MINIMUN)
    return capital

In [ ]:
investment_return(18000, time, yield_)

In [ ]:
def predict_by_year(initial_capital, vehicles):
    capital = initial_capital
    days = 0
    time_serie = []
    for i in vehicles:
        invret = investment_return(capital, i['time'], i['yield'])
        days += i['time']
        capital += invret['return']
        complement = fix_capital(capital)
        time_serie.append({
            "time": time,
            "extra_money": complement,
            "return": invret
        })
        capital += complement
    extra_money = sum([i['extra_money'] for i in time_serie])
    return {
        "time_serie": time_serie,
        "return": capital,
        "days": days,
        "extra_money": extra_money,
        "investment_capital": extra_money + initial_capital,
        "profits": capital - extra_money - initial_capital
    }

In [ ]:
vehicles = [{
    "time": 119,
    "yield": 0.18
},
    {
    "time": 119,
    "yield": 0.18
},
    {
    "time": 119,
    "yield": 0.18
},
]
prediction = predict_by_year(10000, vehicles)
prediction

In [ ]:
10000*0.18

In [ ]:
vehicles = [{
    "time": 90,
    "yield": 0.18
},
    {
    "time": 90,
    "yield": 0.18
},
    {
    "time": 90,
    "yield": 0.18
},
    {
    "time": 90,
    "yield": 0.18
},
]
prediction = predict_by_year(10000, vehicles)
prediction

In [ ]:
vehicles = [{
    "time": 365,
    "yield": 0.18
}]
prediction = predict_by_year(12000, vehicles)
prediction

In [ ]:
2020.651679999999 - 1909.9310399999995